In [28]:
import os
import pandas as pd
import json

In [29]:
# 폴더에 있는 모든 파일을 부른다
path_dir = "hard_csv/"
file_list = [path_dir + file for file in os.listdir(path_dir) if file.endswith(".json")]
file_list

['hard_csv/65.000.json',
 'hard_csv/65.830_2.json',
 'hard_csv/65.830_1.json',
 'hard_csv/65.830_3.json',
 'hard_csv/66.250.json',
 'hard_csv/66.670_1.json',
 'hard_csv/66.670_2.json',
 'hard_csv/66.670_3.json',
 'hard_csv/67.080.json',
 'hard_csv/67.920_1.json',
 'hard_csv/67.920_2.json',
 'hard_csv/68.330_1.json',
 'hard_csv/68.330_2.json',
 'hard_csv/69.170_1.json',
 'hard_csv/69.170_2.json',
 'hard_csv/predictions_fold1.json',
 'hard_csv/predictions_fold2.json',
 'hard_csv/predictions_fold3.json',
 'hard_csv/predictions_fold4.json',
 'hard_csv/predictions_fold5.json',
 'hard_csv/predictions_ng1.json',
 'hard_csv/predictions_ng2.json',
 'hard_csv/predictions_ng3.json',
 'hard_csv/predictions_ng4.json',
 'hard_csv/predictions_ng5.json',
 'hard_csv/predictions_ngqg1.json',
 'hard_csv/predictions_ngqg2.json',
 'hard_csv/predictions_ngqg3.json',
 'hard_csv/predictions_ngqg4.json',
 'hard_csv/predictions_ngqg5.json']

In [30]:
# json list 형태 저장
json_list = []
for file in file_list:
    with open(file, 'r') as f:
        json_list.append(json.load(f))
print(len(json_list))

30


In [31]:
with open('hard_csv/69.170_1.json', 'r') as f:
    em69170_1 = json.load(f)
with open('hard_csv/69.170_2.json', 'r') as f:
    em69170_2 = json.load(f)
with open('hard_csv/68.330_1.json', 'r') as f:
    em68330_1 = json.load(f)
with open('hard_csv/68.330_2.json', 'r') as f:
    em68330_2 = json.load(f)
with open('hard_csv/67.920_1.json', 'r') as f:
    em67920_1 = json.load(f)
with open('hard_csv/67.920_2.json', 'r') as f:
    em67920_2 = json.load(f)
with open('hard_csv/67.080.json', 'r') as f:
    em67080 = json.load(f)
with open('hard_csv/66.670_1.json', 'r') as f:
    em66670_1 = json.load(f)
with open('hard_csv/66.670_2.json', 'r') as f:
    em66670_2 = json.load(f)
with open('hard_csv/66.670_3.json', 'r') as f:
    em66670_3 = json.load(f)
with open('hard_csv/66.250.json', 'r') as f:
    em66250 = json.load(f)
with open('hard_csv/65.830_1.json', 'r') as f:
    em65830_1 = json.load(f)
with open('hard_csv/65.830_2.json', 'r') as f:
    em65830_2 = json.load(f)
with open('hard_csv/65.830_3.json', 'r') as f:
    em65830_3 = json.load(f)
with open('hard_csv/65.000.json', 'r') as f:
    em65000 = json.load(f)

In [32]:
fold_ingredients = []

file_names = [
    'hard_csv/predictions_ngqg1.json', 
    'hard_csv/predictions_fold1.json',
    'hard_csv/predictions_ng1.json'
]

for name in file_names:
    for i in range(1, 6):
        index = name.find('.') - 1
        # print(name[index])
        name = name[:index] + str(i) + name[index+1:]
        # print(name, index)
        with open(name, 'r') as f:
            fold_ingredients.append(json.load(f))

print(len(fold_ingredients))

json_list = [em69170_1, em67920_1, em66670_1, em66250, em65830_2, em65830_3]
json_list.extend(fold_ingredients)

15


In [33]:
# 정답 key list 저장
key_list = list(json_list[0].keys())
len(key_list)

600

In [34]:
# data를 합친다
# best가 data0에 들어가있음(현재)
df_list = []
column_name_list = []
for i, json in enumerate(json_list):
    df = pd.DataFrame([json])
    df = df.transpose()
    column_name = 'data'+str(i)
    df.rename(columns={0:column_name},inplace=True)
    df_list.append(df)
    column_name_list.append(column_name)
    
merge_df = pd.concat(df_list,axis=1)

In [35]:
merge_df

,data0,data1,data2,data3,data4,data5,data6,data7,data8,data9,...,data11,data12,data13,data14,data15,data16,data17,data18,data19,data20
mrc-1-000653,40억년전 지구,별하늘계의 변경에 위치한 외계행성 사만(サマーン),지옥의 유령,사락사라,사락사라,40억년전 지구,지옥,지구,지구,사락사라,...,40억년전,40억년전 지구,사만 행성,사락사라,사락사라,지구,사만 행성,사만 행성,사락사라,지구
mrc-1-001113,냉전,냉전,냉전 종식,냉전 종식,냉전 종식,냉전 종식,냉전,냉전 종식,냉전 종식,냉전 종식,...,냉전 종식,냉전 종식,냉전 종식,냉전,냉전 종식,냉전,냉전,냉전 종식,냉전 종식,냉전
mrc-0-002191,대통령인 빌헬름 미클라스,대통령인 빌헬름 미클라스,대통령인 빌헬름 미클라스,대통령인 빌헬름 미클라스,빌헬름 미클라스,대통령인 빌헬름 미클라스,대통령인 빌헬름 미클라스,빌헬름 미클라스,대통령인 빌헬름 미클라스,대통령인 빌헬름 미클라스,...,대통령인 빌헬름 미클라스,대통령인 빌헬름 미클라스,대통령인 빌헬름 미클라스,대통령인 빌헬름 미클라스,빌헬름 미클라스,대통령인 빌헬름 미클라스,빌헬름 미클라스,빌헬름 미클라스,빌헬름 미클라스,빌헬름 미클라스
mrc-0-003951,뉴질랜드,뉴질랜드,뉴질랜드,뉴질랜드,뉴질랜드,뉴질랜드,뉴질랜드,뉴질랜드,뉴질랜드,뉴질랜드,...,뉴질랜드,뉴질랜드,뉴질랜드,뉴질랜드,뉴질랜드,뉴질랜드,뉴질랜드,뉴질랜드,뉴질랜드,뉴질랜드
mrc-1-001272,프랑스,프랑스,프랑스,프랑스,프랑스군,프랑스,프랑스,프랑스군,프랑스,프랑스,...,프랑스군,프랑스,프랑스,프랑스,프랑스,프랑스,프랑스,프랑스,프랑스군,프랑스
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mrc-0-002989,재단 X,재단 X,재단 X,재단 X,재단 X,재단 X,재단 X,재단 X,재단 X,재단 X,...,재단 X,재단 X,재단 X,재단 X,재단 X,재단 X,재단 X,재단 X,재단 X,재단 X
mrc-0-001804,자유당계 인사들,자유당계 인사들,자유당계 인사들,자유당계 인사들,자유당계 인사들,자유당계 인사들,자유당계 인사들,자유당계 인사들,자유당계 인사들,자유당계 인사들,...,자유당계 인사들,자유당계 인사들,자유당계 인사들,자유당계 인사들,4.19 관련자,자유당계 인사들,자유당계 인사들,자유당계 인사들,자유당계 인사들,자유당계 인사
mrc-0-003411,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임,...,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임,서스펜디드 게임
mrc-0-003436,디에프 항,디에프 항,주노 해변,195고지,주노 해변,주노 해변,주노 해변,주노 해변,주노 해변 주노 해변,주노 해변,...,디에프 항,주노 해변,주노 해변,주노 해변,주노 해변 주노 해변,주노 해변,주노 해변,베리에르 능선,주노 해변,디에프 항


In [10]:
column_name_list

['data0',
 'data1',
 'data2',
 'data3',
 'data4',
 'data5',
 'data6',
 'data7',
 'data8',
 'data9',
 'data10',
 'data11',
 'data12',
 'data13',
 'data14',
 'data15',
 'data16',
 'data17',
 'data18',
 'data19',
 'data20',
 'data21',
 'data22',
 'data23',
 'data24',
 'data25',
 'data26',
 'data27',
 'data28',
 'data29']

In [36]:
from collections import defaultdict

def count_list(data_list):
    # 개수를 key로 하여 data를 저장할 df
    count_df = defaultdict(list)
    
    # 중복 제거 list
    drop_duplicate_list = list(set(data_list))

    for element in drop_duplicate_list:
        data_cnt = data_list.count(element)
        # try:
        count_df[data_cnt].append(element)
        # except:
        #     count_df[data_cnt]=[element]
    
    return count_df

In [12]:
from datasets import load_from_disk, concatenate_datasets
from konlpy.tag import Mecab 
from tqdm.auto import tqdm
tokenizer = Mecab() 


def analysis_data():
    """
    train, validation answer의 형태소별 빈도수 저장
    """
    dataset = load_from_disk('../../data/train_dataset')
    train_dataset = dataset['train']
    val_dataset = dataset['validation']

    merged_dataset = concatenate_datasets([
        train_dataset.flatten_indices(),
        val_dataset.flatten_indices(),
    ])

    # print(merged_dataset.column_names)
    
    freq = defaultdict(int)
    for idx in tqdm(range(len(merged_dataset))):
        # print(merged_dataset['answers'][idx])

        results = tokenizer.pos(merged_dataset['answers'][idx]['text'][0])

        for result in results:
            freq[result[1]] += 1
    
    return freq
    
freq = analysis_data()

Loading cached processed dataset at ../../data/train_dataset/train/cache-fbc57aa6e699fb0c.arrow
Loading cached processed dataset at ../../data/train_dataset/validation/cache-d2fba0c42123b1d6.arrow


KeyboardInterrupt: 

In [21]:
import pickle
with open('freq.pkl', 'wb') as handle:
    pickle.dump(freq, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('freq.pkl', 'rb') as handle:
    freq = pickle.load(handle)


In [37]:
J_list = ['JKS', 'JKS',	'JKC', 'JKC', 'JKG', 'JKG', 'JKO', 'JKO', 'JKB', 'JKB', 'JKV', 'JKV', 'JKQ', 'JKQ', 'JX', 'JC']
# J_freq = []
# for j in J_list: 
#     J_freq.append(freq[j])

# print(J_freq, sum(J_freq))

In [38]:
import numpy as np
def select_morpheme(answer_candidates):
    poses = [tokenizer.pos(answer) for answer in answer_candidates]
    # print(poses)
    answer = []
    
    for pos in poses:
        temp = []
        for p in pos:
            if p[-1] in J_list:
                temp.append(1)
            else:
                temp.append(0)
        answer.append(temp)
                
    # print(answer)
    # [[('시트로엥', 'NNP'), ('자동차', 'NNG'), ('회사', 'NNG')], [('시트로엥', 'NNP')]]
    # [[0, 0, 0], [0]]
    sum_first, sum_last = 0, 0

    for a in answer:
        sum_first += a[0]
        sum_last += a[-1]

    # 모든 답의 앞, 뒤에 조사가 포함된다면
    if sum_first == len(answer) or sum_last == len(answer):
        return answer_candidates[0]

    # 조사가 없는 것 먼저 반환
    for idx, a in enumerate(answer):
        if a[0] == 0 and a[-1] == 0:
            return a

In [39]:
"""
Hard Voting
"""

hard_voting = {}
# key = doc name
for key in key_list:
    # data 저장
    data_list = []
    for col in column_name_list:
        data_list.append(merge_df.loc[key][col])
    # data count
    count = count_list(data_list)

    answer_candidates = count[max(count.keys())]
    # print(answer_candidates)
    # 빈도수 동률 발생
    if len(answer_candidates) > 1:
        hard_voting[key] = select_morpheme(answer_candidates)
    elif len(answer_candidates) == 1:
        hard_voting[key] = answer_candidates[0]

In [40]:
hard_voting

{'mrc-1-000653': '사락사라',
 'mrc-1-001113': '냉전 종식',
 'mrc-0-002191': '대통령인 빌헬름 미클라스',
 'mrc-0-003951': '뉴질랜드',
 'mrc-1-001272': '프랑스',
 'mrc-1-000993': '아래턱',
 'mrc-0-005021': '대의제도',
 'mrc-1-000163': '하코네 산의 화산 폭발',
 'mrc-0-001283': '순조 11년(1811)',
 'mrc-0-004543': '고전도성 철',
 'mrc-0-000439': '리처드 말킨',
 'mrc-0-002895': '칼라치 전방 약250km 지점',
 'mrc-0-000535': '롭 포드',
 'mrc-1-001724': '노스햄프턴 교회',
 'mrc-0-000901': '언니',
 'mrc-0-001606': '석조 궁륭',
 'mrc-0-000266': '국민',
 'mrc-0-001326': '‘자유’지에',
 'mrc-0-000032': '국방부 보고서',
 'mrc-0-005215': '입석대',
 'mrc-0-005407': '숙의 정씨',
 'mrc-0-003683': '국가인권위원회',
 'mrc-0-003644': '이탈리아',
 'mrc-0-002835': '망자',
 'mrc-0-000049': '서사',
 'mrc-1-001829': '화흡',
 'mrc-1-001662': '소마 나카무라 번의 배신',
 'mrc-0-001206': '2004년',
 'mrc-0-004007': '신시내티대학교',
 'mrc-1-000418': '일등바위',
 'mrc-0-003133': '불법 어업',
 'mrc-0-004646': '건늠선',
 'mrc-0-001058': '카필라 성',
 'mrc-0-002361': '독립',
 'mrc-0-004830': '13개',
 'mrc-0-002762': '경복사',
 'mrc-0-000395': '도우카즈토시',
 'mrc-0-001668': '어머

In [42]:
import json
with open('prediction.json', 'w') as handle:
    json.dump(hard_voting, handle)